### IntSet with Assertions (Java) [10 points]

The Java implementation of `IntSet` below has the capacity as a parameter of the constructor rather than a constant. Add assertions to check the method precondition, class invariants, and loop invariants. Add method `intSetInvariantOK` to `IntSet` to check its invariant and method `maxIntSetInvariantOK` to `MaxIntSet` to check its invariant, such that the check in `MaxIntSet` includes the check of `IntSet` (but not the other way round). Add also checks for the loop invariants; you may introduce additional methods if that shortens your solution. The pattern for checking loop invariants is:
```Java
    assert loopInvariantOK;
    while (guard) {
        body; assert loopInvariantOK;
    }
```
Often, assertion checks consist of several conditions, such as in:
```Java
    boolean invariantOK() {return A && B;}
```
To see which condition is violated, one can instead write
```Java
    boolean invariantOK() {assert A; assert B; return true;}
```
or:
```Java
    boolean invariantOK() {assert A; return B;}
```
This way, the message for an assertion violation will point to the failed assert statement. Use this style in this question.

Note that the two methods for class invariant checking are given different names: if they had the same name, say `invariantOK()`, a call to that method in `IntSet` (at the end of another method) would go to `invariantOK()` of `MaxIntSet` due to the dynamic binding. That is not correct; methods of `IntSet` should only check the invariant of `IntSet`, as these may be called within methods of `MaxIntSet` (or some other class that extends `IntSet`) where the invariant of `MaxIntSet` is not expected to hold. Recall that invariants have to hold only at the beginning and end of methods, not in between.

You can add test cases as you like; they are not being graded.

In [2]:
%%writefile intset.java
class IntSet {
    int a[];
    int n;
    boolean intSetInvariantOK() {
        // return true if 0 ≤ n ≤ a.length ∧ (∀ i ∈ 0 .. n - 1, j ∈ i + 1 .. n - 1 · a[i] ≠ a[j])
        // otherwise fail
    }
    IntSet(int capacity) {
        a = new int[capacity]; n = 0;
    }
    void add(int x) {
        int i = 0;
        while (i < n && a[i] != x) {
            i += 1;
        }
        if (i == n) {
            a[n] = x; n += 1;
        }
    }
    boolean has(int x) {
        int i = 0;
        while (i < n && a[i] != x) {
            i += 1;
        }
        return i < n;
    }
}
class MaxIntSet extends IntSet {
    int m;
    boolean maxIntSetInvariantOK() {
        assert intSetInvariantOK();
        // return true if n > 0 ⇒ m = max(a[0 .. n - 1])
        // otherwise fail
        ...
    }
    MaxIntSet(int capacity) {
        super(capacity);
    }
    void add(int x) {
        super.add(x);
        if (n == 1) m = x;
        else m = m > x ? m : x;
    }
    int maximum(){
        return m;
    }
}
class TestMaxIntSet {
    public static void main(String[] args) {
        MaxIntSet s = new MaxIntSet(3); s.add(5); s.add(7);
        System.out.println(s.maximum());
    }
}

Overwriting intset.java


In [34]:
%%writefile intset.java
class IntSet {
    int a[];
    int n;
    boolean intSetInvariantOK() {
        // return true if 0 ≤ n ≤ a.length ∧ (∀ i ∈ 0 .. n - 1, j ∈ i + 1 .. n - 1 · a[i] ≠ a[j])
        // otherwise fail
        for (int i = 0; i < n; i++){
            for (int j = i+1; j < n; j++){
                if(this.a[j] == this.a[i]) return false;
            }
        }
        return 0 <= this.n && this.n <= this.a.length;
    }

    IntSet(int capacity) {
        assert capacity > 0;
        a = new int[capacity]; n = 0;
        assert intSetInvariantOK();
    }

    boolean exists_LoopInvariant(int i, int x){
        for (int j = 0; j < i; j++) {
            if (a[j]==x) return false;
        }
        return true; 
    }

    void add(int x) {
        assert n < a.length;
        int i = 0;
        assert exists_LoopInvariant(i, x);
        while (i < n && a[i] != x) {
            i += 1; exists_LoopInvariant(i, x);
        }
        if (i == n) {
            a[n] = x; n += 1;
        }
        assert intSetInvariantOK();
    }

    boolean has(int x) {
        int i = 0;
        assert exists_LoopInvariant(i, x);
        while (i < n && a[i] != x) {
            i += 1; assert exists_LoopInvariant(i, x);
        }
        assert intSetInvariantOK();
        return i < n;
    }
}
class MaxIntSet extends IntSet {
    int m;
    boolean maxIntSetInvariantOK() {
        assert intSetInvariantOK();
        // return true if n > 0 ⇒ m = max(a[0 .. n - 1])
        // otherwise fail

        if (n > 0){ 
            int m = a[0]; 
            for (int i = 1; i < n; i++) {
                if (a[i] > this.m) {
                    return false;
                }
            }
        }

        return true;
    }
    MaxIntSet(int capacity) {
        super(capacity);
    }
    void add(int x) {
        super.add(x);
        if (n == 1) m = x;
        else m = m > x ? m : x;
        assert maxIntSetInvariantOK();
    }
    int maximum(){
        return m;
    }
}
class TestMaxIntSet {
    public static void main(String[] args) {
        MaxIntSet s = new MaxIntSet(3); s.add(5); s.add(7);
        System.out.println(s.maximum());
    }
}

Overwriting intset.java


In [35]:
!javac intset.java

In [36]:
!java -ea TestMaxIntSet

7


What is the computational effort of checking the class and loop invariants compared to the actual computation?

checking the class and loop invariant significantly adds to the computational efforts. this is because the loops in add() and has() have increased in complexity; from O(n) to O(n^2) since for each iteration of the loop we recheck all the previous elements again. the intSetInvariantOK() also has a complexity of O(n^2) since we compare each element of the array with every other element. we call this invariant many times in the class which adds to the computational effort.